# Step 1 - Project setup

This notebook creates a project setup file with all the user's project's information.

The project setup file prepares the locations for all files relevant to the project, and sets project parameters.

written by Andries Heyns

email aheyns@worldbank.org

In [ ]:
##################################################################
##################################################################

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from geopandas.tools import sjoin
import geojson
import shutil
import matplotlib.path as mplPath
from shapely.geometry import Point
from shapely.geometry import Polygon
from datetime import datetime
import json
import rasterio
from rasterio.transform import Affine
import numpy as np

##################################################################
##################################################################
#set up project parameters
#your project folder
data_root = 'D:\\github_test\\'

##################################################################
#final CRS, change to your region.
dest_crs = 'EPSG:32642'
dest_crs_id = '32642'

##################################################################
#buffer distance in metres, if desired
buffer_m = 2000 #example, 2000 for 2km

##################################################################
#administrative level of analysis extent
#select which the LARGEST administrative level that you are investigating
#example, if you are investigating adm2 and adm3 within a Province (adm1)
#then choose adm1

#use "custom" if you have a custom boundary

#NOTE: Currently, the code is not yet fully generic and is set up for
#adm2 and adm3 analyses within adm1 boundaries only
#future versions will be more modular

# level = 'adm0' #for a country
level = 'adm1' #for provinces
# level = 'adm2' #for districts/regions
# level = 'adm3' #for sub-districts/sub-regions (tehsils in Pakistan)

# level = 'custom' #your own custom area

##################################################################
#specify the field name in your shapefile and the name of the administrative unit you are investigating
#for finding the correct boundary for population weighting and aggregation
shapefile_adm_field = 'ADM1_EN'
adm_name = 'Khyber Pakhtunkhwa'

##################################################################
#max administrative level of analysis
#select the maximum depth of administrative level that you are investigating
#example, if you are investigating up to adm3 within a Province (adm1)
#then choose adm3

#NOTE: This is not relevant to custom boundaries
#Currently, the code is not yet fully generic and is set up for
#either a max depth of adm2 or adm3 analyses within adm1 boundaries only
#future versions will be more modular

# max_level = 'adm1' #for provinces
# max_level = 'adm2' #for districts/regions
max_level = 'adm3' #for sub-districts/sub-regions (tehsils in Pakistan)

##################################################################
#force use of rivers, despite google earth dynamic world datasets already containing seasonal rivers
#1 for yes, 0 for no
force_rivers = 0 

##################################################################
#do you have seasonal land cover datasets or a single land cover dataset?
#if you don't have seasonal datasets you will have to provide a year-round dataset from e.g. ICIMOD / ESA WorldCover 
#and specify seasonal walking speed reduction values
#1 for yes, 0 for no
seasonal_lc = 1

##################################################################
##################################################################
#set up all project folders, and additional information used later
#avoid duplicating the structure creation if already done - in future, warn user and suggest a different project name if it's for a new project
if os.path.isdir(data_root):
    print('project folder already exists') 
else:
    fric_dir = data_root + 'friction\\'
    fric_dir_onroad = data_root + 'friction\\onroad\\'
    access_dir = data_root + 'access\\'
    inputs_dir = data_root + 'inputs\\'
    local_dem_folder = data_root + 'inputs\\elevation\\'
    local_population_folder = data_root + 'inputs\\population\\'
    local_lc_folder = data_root + 'inputs\\landcover\\'
    local_lc_folder_non_seasonal = data_root + 'inputs\\landcover\\Non_seasonal\\'
    local_waterways_folder = data_root + 'inputs\\water\\'
    local_roads_folder = data_root + 'inputs\\roads\\'
    local_destination_folder = data_root + 'inputs\\destinations\\'
    local_boundaries_folder = data_root + 'inputs\\boundaries\\'
    local_boundaries_folder_adm0 = data_root + 'inputs\\boundaries\\adm0\\'
    local_boundaries_folder_adm1 = data_root + 'inputs\\boundaries\\adm1\\'
    local_boundaries_folder_adm2 = data_root + 'inputs\\boundaries\\adm2\\'
    local_boundaries_folder_adm3 = data_root + 'inputs\\boundaries\\adm3\\'  
    local_boundaries_folder_custom = data_root + 'inputs\\boundaries\\custom\\' 
    pixels_dir = access_dir + 'pixels\\'
    tables_dir = access_dir + 'tables\\'
    tables_separate_dir = tables_dir + 'separate\\'

    os.mkdir(data_root)
    os.mkdir(inputs_dir)
    os.mkdir(fric_dir)
    os.mkdir(fric_dir_onroad)
    os.mkdir(access_dir)
    os.mkdir(pixels_dir)
    os.mkdir(tables_dir)
    os.mkdir(tables_separate_dir)
    os.mkdir(local_dem_folder)
    os.mkdir(local_population_folder)
    os.mkdir(local_lc_folder)
    os.mkdir(local_lc_folder_non_seasonal)
    os.mkdir(local_waterways_folder)
    os.mkdir(local_destination_folder)
    os.mkdir(local_boundaries_folder)
    os.mkdir(local_boundaries_folder_adm0)
    os.mkdir(local_boundaries_folder_adm1)
    os.mkdir(local_boundaries_folder_adm2)
    os.mkdir(local_boundaries_folder_adm3)
    os.mkdir(local_boundaries_folder_custom)
    os.mkdir(local_roads_folder)

##################################################################
##################################################################
#write final project setup file

data_set = {
    "data_root":data_root,
    "buffer_m":buffer_m,
    "dest_crs":dest_crs,
    "dest_crs_id":dest_crs_id,
    "level":level,
    "max_level":max_level,
    "seasonal_lc":seasonal_lc,
    "shapefile_adm_field":shapefile_adm_field,
    "adm_name":adm_name,
    "force_rivers":force_rivers,
    "local_dem_folder":local_dem_folder,
    "local_population_folder":local_population_folder,
    "local_lc_folder":local_lc_folder,
    "local_lc_folder_non_seasonal":local_lc_folder_non_seasonal,
    "local_waterways_folder":local_waterways_folder,
    "local_roads_folder":local_roads_folder,
    "local_destination_folder":local_destination_folder,
    "local_boundaries_folder":local_boundaries_folder,
    "local_boundaries_folder_adm0":local_boundaries_folder_adm0,
    "local_boundaries_folder_adm1":local_boundaries_folder_adm1,
    "local_boundaries_folder_adm2":local_boundaries_folder_adm2,
    "local_boundaries_folder_adm3":local_boundaries_folder_adm3,
    "local_boundaries_folder_custom":local_boundaries_folder_custom,
    "fric_dir":fric_dir,
    "fric_dir_onroad":fric_dir_onroad,
    "access_dir":access_dir,
    "pixels_dir":pixels_dir,
    "tables_dir":tables_dir,
    "tables_separate_dir":tables_separate_dir
}

# Define the file path for the JSON file
file_path = data_root+'project_data.json'
# Write the data to the JSON file
with open(file_path, "w") as json_file:
    json.dump(data_set, json_file, indent=4)

print("Folders prepared successfully.")